In [ ]:
import osmnx as ox
from pathlib import Path
import os
import geopandas as gpd
import matplotlib.pyplot as plt

from mclights.utils.get_data import get_data_osmx_or_local

project_dir = Path(os.getcwd()).parent
data_dir = project_dir / "data"
temp_data_dir = project_dir / "temp_data"
raw_data_dir = project_dir / "raw_data"

voivodships_names = [
    "Dolnośląskie",
    "Kujawsko-Pomorskie",
    "Lubelskie",
    "Lubuskie",
    "Łódzkie",
    "Małopolskie",
    "Mazowieckie",
    "Opolskie",
    "Podkarpackie",
    "Podlaskie",
    "Pomorskie",
    "Śląskie",
    "Świętokrzyskie",
    "Warmińsko-Mazurskie",
    "Wielkopolskie",
    "Zachodniopomorskie"
]

In [ ]:
voivodeships_path = data_dir / "voivodeships.gpkg"

tags={"boundary": "administrative","admin_level": "4"}
voivodeships = get_data_osmx_or_local(raw_data_dir / "voivodeships.gpkg", tags)

voivodeships = voivodeships.loc[["relation"], ["name", "geometry"]]
voivodeships["name"] = voivodeships["name"].str.title()
voivodeships = voivodeships[voivodeships["name"].str.contains("|".join(voivodships_names), na=False)]
voivodeships = voivodeships[~voivodeships["name"].str.contains("Urząd")]
voivodeships = voivodeships.reset_index(drop=True)

voivodeships.to_file(voivodeships_path, layer=voivodeships_path.stem, driver="GPKG")

In [ ]:
cities_path = data_dir / "cities.gpkg"

cities = get_data_osmx_or_local(raw_data_dir / "cities_raw.gpkg", {"place": ["city", "town", "village"]})

cities = cities.loc["node", ["geometry", "name", "place", "population", "name:pl", "wikipedia"]].reset_index(drop=True)
cities["name"] = (
    cities["name"]
        .fillna(cities["name:pl"])
        .fillna(cities["wikipedia"].str.extract(r"^pl:(.*?)\s*\(", expand=False))
)
cities = cities.dropna(subset=["name"])
cities = cities.drop(columns=["name:pl", "wikipedia"])

cities = gpd.sjoin(
    cities,
    voivodeships.rename(columns={"name": "voivodeship"}),
    how="left",
    predicate="within"
)

cities.to_file(cities_path, layer=cities_path.stem, driver="GPKG")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

voivodeships.plot(
    ax=ax,
    facecolor="none",
    edgecolor="black",
    linewidth=1
)

plt.show()